In [1]:
import requests
import pandas as pd
import os
%load_ext dotenv
%dotenv keys.env

In [2]:
api_key = os.getenv('electronicKey') # API key stored in .env file
params = {
    'apiKey': api_key, 
    'format': 'json',
    } 

headers = { 
    'Content-Type' : 'application/json',
    'Authorization' : 'apikey '+api_key
    } 

In [20]:
df = pd.read_excel('.xlsx', dtype=str).to_dict('records') # read in the excel file

In [ ]:
for row in df:
    collection_id = row['Electronic Collection Id']
    portfolio_id = row['Portfolio Id']
    service_id = row['Service Id']

    url = f'https://api-na.hosted.exlibrisgroup.com/almaws/v1/electronic/e-collections/{collection_id}/e-services/{service_id}/portfolios/{portfolio_id}' # url for the API call

    response = requests.get(url, params=params) # API call
    data = response.json() # convert the response to json

    try:
        data['linking_details']['static_url'] = data['linking_details']['static_url'].replace('http://prox.miracosta.edu/login?url=', '') # remove the proxy from the static url
        a = requests.put(url, json=data, headers=headers) # update the portfolio
        print(f'Portfolio: {portfolio_id}, {a}') # print the response
        log = open('log.txt', 'a') # log the response
        log.write(f'Portfolio: {portfolio_id}, {a} \n') 
        log.close()

    except KeyError as error:
        f = open('error_log.txt', 'a')
        f.write(f'Portfolio: {portfolio_id}, error: {error} \n')     
        print(f'Portfolio: {portfolio_id}, error: {error}')
        f.close()